Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In [8]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    secret_share = protocol.uniform(shape=(2,2))
    secret = protocol.encoder.decode(protocol.reveal(secret_share))
    log.info(f"Player {communicator.rank} secret: \n{secret}")
    absolute_share = protocol.absolute(secret_share)
    absolute = protocol.encoder.decode(protocol.reveal(absolute_share))
    log.info(f"Player {communicator.rank} absolute(secret): \n{absolute}")
    


In [ ]:
main()

In this example, we generate a 2x2 matrix of values selected uniformly at random from the field. Then we will reveal them so we can check correctness of the protocol.

Next we will call the absolute method on the secret_share which will calculate the ablsolute value of the matrix on an elementwise basis and return the shared result. This we reveal and decode and print to check against what we would expect given the original secrets.

As you can see from inspection the result of the protocol satifies the desired functionality of what one would expect from absolute value.